In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

###### Load the data into dataframe

In [49]:
data=pd.read_csv("merged.csv")

In [54]:
data2=pd.read_csv("merged.csv")
len(data2)

2270829

###### First Thing we will try to remove all the row in which the  price feature is irrrelevent as we have discussed in EDA above.


In [51]:
data.drop(data[data.price >150000].index,inplace=True)
data.drop(data[data.price <500].index,inplace=True)


###### Lets remove the vim, Size, url ,lat ,long and image_url features 

In [58]:
data=data.drop(['vin','size','lat','long','url','image_url'],axis=1)

In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2162326 entries, 0 to 2270827
Data columns (total 14 columns):
city            object
price           int64
year            float64
manufacturer    object
make            object
condition       object
cylinders       object
fuel            object
odometer        float64
title_status    object
transmission    object
drive           object
type            object
paint_color     object
dtypes: float64(2), int64(1), object(11)
memory usage: 247.5+ MB


###### Replacing the paint_color,title_status nan with unknown
As if we specify any color then it might affect the model

In [62]:
data['paint_color'].fillna("unknown",inplace =True)
data['title_status'].fillna("unknown",inplace =True)

###### Now we are removing the rows that dont have year of manufacturing as they are aproximately 7000 and we can't substitute them 

In [63]:
data.drop(data[data.year.isna()].index,inplace=True)

In [64]:
len(data)

2162326

###### Now we will try to remove the rows whose year is before 1990 
As we know from EDA that we have data of year past 1990 .We also have some rows which has year as 302,718,... which are obviously invalid. 
As these car are considered as vintage cars ,whose price is mostly unpredictiable depending on their features.Their price really depend on how much close they remain to the original manufacturer's design,features and specification.Vintage Car's value affects due to reasons like any sort of change and modification to car in fact,  the car may even lose price if it goes through too many alterations.[ref from www.coverfox.com ] 
We almost loose 100k of rows but most of those are dirty data

In [94]:
data.drop(data[data.year <1990].index,inplace=True)
data.drop(data[data.year >=2020].index,inplace=True)

In [95]:
len(data)-data.count()

city                 0
price                0
year                 0
manufacturer    123841
make             61118
condition       829447
cylinders       799512
fuel             13790
odometer        538367
title_status         0
transmission     11997
drive           715409
type            742784
paint_color          0
dtype: int64

#### Now we will to remove NaN values from Manufacturer feature

Manufacturer column can easily be identified by make features.
But there are some car that have same make but are of different manufacturers which are very few in numbers.  
##### Here are such cars With their year of launch:
Jeep XJ(1983) and Jaguar XJ(2013)  
Pontiac GTO(1964) and Mitsubishi GTO(1990) and Ferrari GTO(1962)  
Edsel Ranger(1960) and Ford Ranger(2001)  
Edsel Citation(1958) and Chevrolet Citation(1980)  
Dodge Lancer(1995) and Mitsubishi Lancer(1973)  
Dodge Aspen(1976) and Chrysler Aspen(2007)  
Chevrolet Suburban(Yet to launch) and Plymouth Suburban(1949)  
Ford Sierra(1982) and GMC Sierra(2010) 

As we can see that these car have a fat amount of years between them so there is a rare situation that we can get both cars we can also testify it as below

In [96]:
data[data['make']=='subrban']['manufacturer'].unique()

array(['chevrolet'], dtype=object)

In [97]:
data[data['make']=='gto']['manufacturer'].unique()


array(['pontiac'], dtype=object)

###### removing rows that contain 'model' as make they are around 969

In [107]:
data.drop(data[data.make =='model'].index,inplace=True)

In [108]:
test=data.copy(deep=True)

###### Replacing nan values now

In [109]:
test['manufacturer']=test['manufacturer'].fillna('50')


In [110]:
rrr=test[test['manufacturer']=='50']
(test['manufacturer']=='50').sum()

122874

In [102]:
def man(ddd):
    if ddd['manufacturer']!='50':
        return
    #manu=ddd['manufacturer']
    maker=ddd['make']
    #print(manu)
    fff=test[( (test['make']==maker) & (test['manufacturer']!='50'))].head(1)
    if fff.empty:
        test.loc[((test['manufacturer']=='50')&(test['make']==maker)),'manufacturer']='51'
        pass
    else:
        tt=fff.iloc[0]
        test.loc[((test['make']==maker)& (test['manufacturer']=='50')),'manufacturer']=tt['manufacturer']
    return
rrr.apply(man,axis=1)

KeyboardInterrupt: 

In [106]:
(test['make']=='model').sum()

969

In [ ]:
test['manufacturer']=test['manufacturer'].replace('51','unknown')
test['manufacturer']=test['manufacturer'].replace('50','unknown')

In [ ]:
test.to_csv("cleanedPart1.csv",index=False)

In [105]:
(test['manufacturer']=='model').sum()

0

 # Let's try to remove NaN values from Fuel Feature
 So in order to fill nan values for fuel we need to concentrate on following columns 
 Manufacturer,
 Make.
 AS if we know the make and manufacturer of that column we can find out the fuel type in our database itself
 

In [1]:
display(test[['year','manufacturer','make','fuel']].head())
display(test['fuel'].unique())
display(test['fuel'].isna().sum())

NameError: name 'data' is not defined

In [ ]:
display(test[(test['fuel'].isna())].head(10))
cleanedf=test.copy(deep=True)

In [ ]:
test['fuel']=test['fuel'].fillna('50')

In [ ]:
rrr=test[test['fuel']=='50']
(test['fuel']=='50').sum()

In [ ]:
rrr

In [ ]:
def man(ddd):
    if ddd['fuel']!='50':
        return
    manu=ddd['manufacturer']
    maker=ddd['make']
    #print(manu)
    fff=test[( (test['make']==maker) & (test['manufacturer']!='unknown') & (test['manufacturer']==manu) & (test['fuel']!='50'))].head(1)
    if fff.empty:
        test.loc[((test['fuel']=='50')&(test['make']==maker)&(test['manufacturer']!='unknown') & (test['manufacturer']==manu)),'fuel']='51'
        #pass
    else:
        tt=fff.iloc[0]
        test.loc[((test['make']==maker) & (test['manufacturer']==manu) & (test['fuel']=='50')&(test['manufacturer']!='unknown')),'fuel']=tt['fuel']
    return
rrr.apply(man,axis=1)

In [ ]:
test['fuel']=test['fuel'].replace('50','unknown')
test['fuel']=test['fuel'].replace('51','unknown')


In [ ]:
len(test)-test.count()

In [ ]:
test.to_csv("cleanedPart2.csv",index=False)

## Now we will to remove NaN values from transmission feature

Tramission column is also can be identified by manufacturer and make features.
But same model of that manufactured can be sold with different transmission variant.So we will only assign the transmission type which has higher number of occurance.


In [ ]:
display(test[['year','manufacturer','make','transmission']].head())
display(test['transmission'].unique())
display(test['transmission'].isna().sum())

In [ ]:
display(test[(test['transmission'].isna())])
cleanedtrans=test.copy(deep=True)

In [ ]:
test['transmission']=test['transmission'].fillna('50')

In [ ]:
rrr=test[test['transmission']=='50']

In [ ]:
rrr

In [ ]:
def man(ddd):
    if ddd['transmission']!='50':
        return
    manu=ddd['manufacturer']
    maker=ddd['make']
    #print(manu)
    fff=test[( (test['make']==maker) & (test['manufacturer']!='unknown') & (test['manufacturer']==manu) & (test['transmission']!='50'))].head(1)
    if fff.empty:
        test.loc[((test['transmission']=='50')&(test['make']==maker)&(test['manufacturer']!='unknown') & (test['manufacturer']==manu)),'transmission']='51'
        #pass
    else:
        tt=fff.iloc[0]
        test.loc[((test['make']==maker) & (test['manufacturer']==manu) & (test['transmission']=='50')&(test['manufacturer']!='unknown')),'transmission']=tt['transmission']
    return
rrr.apply(man,axis=1)

In [ ]:
test['transmission']=test['transmission'].replace('50','unknown')
test['transmission']=test['transmission'].replace('51','unknown')


In [ ]:
len(test)-test.count()

In [ ]:
test.to_csv("cleanedPart3.csv",index=False)

## Now we will to remove NaN values from cylinders  feature

Cylinders column is also can be identified by manufacturer and make features.


In [ ]:
display(test[['year','manufacturer','make','cylinders']].head())
display(test['cylinders'].unique())
display(test['cylinders'].isna().sum())

In [ ]:
display(test[(test['cylinders'].isna())])
cleanedcylin=test.copy(deep=True)

In [ ]:
test['cylinders']=test['cylinders'].fillna('50')

In [ ]:
rrr=test[test['cylinders']=='50']

In [ ]:
rrr

In [ ]:
def man(ddd):
    if ddd['cylinders']!='50':
        return
    manu=ddd['manufacturer']
    maker=ddd['make']
    #print(manu)
    fff=test[( (test['make']==maker) & (test['manufacturer']!='unknown') & (test['manufacturer']==manu) & (test['cylinders']!='50'))].head(1)
    if fff.empty:
        test.loc[((test['cylinders']=='50')&(test['make']==maker)&(test['manufacturer']!='unknown') & (test['manufacturer']==manu)),'cylinders']='51'
        #pass
    else:
        tt=fff.iloc[0]
        test.loc[((test['make']==maker) & (test['manufacturer']==manu) & (test['cylinders']=='50')&(test['manufacturer']!='unknown')),'cylinders']=tt['cylinders']
    return
rrr.apply(man,axis=1)

In [ ]:
test['cylinders']=test['cylinders'].replace('50','unknown')
test['cylinders']=test['cylinders'].replace('51','unknown')


In [ ]:
len(test)-test.count()

In [ ]:
test.to_csv("cleanedPart4.csv",index=False)

## Now we will to remove NaN values from drive  feature

drive column is also can be identified by manufacturer and make features.


In [ ]:
display(test[['year','manufacturer','make','drive']].head())
display(test['drive'].unique())
display(test['drive'].isna().sum())

In [ ]:
display(test[(test['drive'].isna())])
cleaneddrive=test.copy(deep=True)

In [ ]:
test['drive']=test['drive'].fillna('50')

In [ ]:
rrr=test[test['drive']=='50']

In [ ]:
rrr

In [ ]:
def man(ddd):
    if ddd['drive']!='50':
        return
    manu=ddd['manufacturer']
    maker=ddd['make']
    #print(manu)
    fff=test[( (test['make']==maker) & (test['manufacturer']!='unknown') & (test['manufacturer']==manu) & (test['drive']!='50'))].head(1)
    if fff.empty:
        test.loc[((test['drive']=='50')&(test['make']==maker)&(test['manufacturer']!='unknown') & (test['manufacturer']==manu)),'drive']='51'
        #pass
    else:
        tt=fff.iloc[0]
        test.loc[((test['make']==maker) & (test['manufacturer']==manu) & (test['drive']=='50')&(test['manufacturer']!='unknown')),'drive']=tt['drive']
    return
rrr.apply(man,axis=1)

In [ ]:
test['drive']=test['drive'].replace('50','unknown')
test['drive']=test['drive'].replace('51','unknown')


In [ ]:
len(test)-test.count()

In [ ]:
test.to_csv("cleanedPart5.csv",index=False)

## Now we will to remove NaN values from type  feature

type column is also can be identified by manufacturer and make features.


In [ ]:
display(test[['year','manufacturer','make','type']].head())
display(test['type'].unique())
display(test['type'].isna().sum())

In [ ]:
display(test[(test['type'].isna())])
cleanedtype=test.copy(deep=True)

In [ ]:
test['type']=test['type'].fillna('50')

In [ ]:
rrr=test[test['type']=='50']

In [ ]:
rrr

In [ ]:
def man(ddd):
    if ddd['type']!='50':
        return
    manu=ddd['manufacturer']
    maker=ddd['make']
    #print(manu)
    fff=test[( (test['make']==maker) & (test['manufacturer']!='unknown') & (test['manufacturer']==manu) & (test['type']!='50'))].head(1)
    if fff.empty:
        test.loc[((test['type']=='50')&(test['make']==maker)&(test['manufacturer']!='unknown') & (test['manufacturer']==manu)),'type']='51'
        #pass
    else:
        tt=fff.iloc[0]
        test.loc[((test['make']==maker) & (test['manufacturer']==manu) & (test['type']=='50')&(test['manufacturer']!='unknown')),'type']=tt['type']
    return
rrr.apply(man,axis=1)

In [ ]:
test['type']=test['type'].replace('50','unknown')
test['type']=test['type'].replace('51','unknown')


In [ ]:
len(test)-test.count()

In [ ]:
test.to_csv("cleanedPart6.csv",index=False)